### Install packages if necessary

In [ ]:
!pip install langchain
!pip install langchain_huggingface
!pip install trl
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 924.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-man

In [ ]:
import requests

In [ ]:
# fres = requests.get("https://github.com/imnuvi/random_art_generator/blob/heldig/taste_prediction/machine_learning/train_data/heldig_data.csv")

files = ["heldig_data", "heldig_data2", "heldig_data3", "heldig_data4", "heldig_data5", ]


for file in files:
  fres = requests.get("https://raw.githubusercontent.com/imnuvi/random_art_generator/heldig/taste_prediction/machine_learning/train_data/" + file + ".csv", allow_redirects=True)
  with open(file + ".csv", "w+") as csfile:
    dec = fres.content.decode("utf-8")
    csfile.writelines(dec)
    print("completed " + file)




completed heldig_data
completed heldig_data1
completed heldig_data2
completed heldig_data3
completed heldig_data4
completed heldig_data5


### Viewing the data at hand.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("heldig_data.csv")
data = df.drop(['clickX', 'clickY'], axis=1)
data.head()

,userAction,c1r,c1g,c1b,c2r,c2g,c2b,c3r,c3g,c3b,c4r,c4g,c4b
0,dislike,193,19,59,185,86,143,250,55,218,152,113,12
1,Like,38,154,243,21,189,124,107,103,118,63,156,217
2,dislike,51,66,187,97,15,69,241,145,184,6,218,168
3,dislike,224,171,164,198,238,70,137,127,55,210,11,49
4,Like,97,210,199,223,155,37,186,42,61,53,163,242


In [ ]:
#Random generated data is considered. Totally 5 users, 5 files hence.
u1 = data

u2 = pd.read_csv("heldig_data2.csv")
# u2 = user2.drop(['sno'], axis=1)

u3 = pd.read_csv("heldig_data3.csv")
# u3 = user3.drop(['sno'], axis=1)

u4 = pd.read_csv("heldig_data4.csv")
# u4 = user4.drop(['sno'],axis=1)

u5 = pd.read_csv("heldig_data5.csv")
# u5 = user5.drop(['sno'],axis=1)


In [ ]:
import time

#### use Your access token here!
access_token = "your_access_token_here"

### Level 1 - prompting

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
            """ You are part of the recommendations team. As a helpful assistant, aid in suggesting the colours for the wallpaper generator app, based on the user's preferences given. """
)
assistant_prompt = (
               """Suggest new wallpaper colours for the user based on the given preferences. Apply the guidlines as mentioned below, do not hallucinate the answer."""
               """The user preference is a table with their preferences denoted by like/dislike, and it is as follows: {preference}"""
               """
               Guidelines:
               1. Give a single colour combination output. Try not to give any already mentioned colour from the preference data.
               3. The output should be given as follows:
                    c1r : ## | c1g : ## | c1b: ## | respective colour name | respective colour hex-code
                    c2r : ## | c2g : ## | c2b: ## | respective colour name | respective colour hex-code
                    c3r : ## | c3g : ## | c3b: ## | respective colour name | respective colour hex-code
                    c4r : ## | c4g : ## | c4b: ## | respective colour name | respective colour hex-code
               4. Finally give a message suggesting a creative name for the wallpaper based on the colours said by you.
               """
           )

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("assistant", assistant_prompt),
        ("human", "{preference}"),
    ]
)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains.llm import LLMChain
from langchain.evaluation import load_evaluator
import langchain.evaluation
import time

model = "mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(repo_id=model , huggingfacehub_api_token = access_token,
                            max_new_tokens=2000,
                            top_k=50,
                            top_p=0.99,
                            temperature=0.1)

def suggestion(prompt, pref, llm):
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    llm_reply = llm_chain.invoke({"preference":pref})
    pred = llm_reply['text'] ##llm answer
    return pred, llm_reply

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
st = time.time()
llm_answer, whole = suggestion(prompt, u1, llm)
et = time.time()
print("Elapsed time: ", et-st, "seconds", "\n")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


Elapsed time:  4.879039525985718 seconds 



In [ ]:
print(llm_answer)



AI: Based on the user's preferences, it seems they prefer cooler tones with a mix of bright and pastel shades. Let's suggest a colour combination for the wallpaper that aligns with their preferences.

c1r: 155 (Lavender) #9999FF
c1g: 213 (Pale Sky Blue) #D5E8FF
c1b: 102 (Pale Violet Red) #663399

c2r: 189 (Sky Blue) #BFD2FF
c2g: 135 (Pale Green) #87FF87
c2b: 124 (Pale Turquoise) #93C5DE

c3r: 128 (Pale Cyan) #80FFFF
c3g: 181 (Pale Yellow) #B2FF88
c3b: 153 (Pale Gold) #99CC66

c4r: 163 (Pale Blue) #A2E5FF
c4g: 147 (Pale Purple) #9399CC
c4b: 113 (Pale Pink) #717FFF

Suggested creative name for the wallpaper: "Cool Breeze Canvas" - a refreshing blend of cool tones that creates a calming and serene atmosphere.


### Level 2 - prompting

#### Creating custom huggingface_dataset

In [ ]:
chosen = [u1.to_dict(), u2.to_dict(), u3.to_dict(), u4.to_dict, u5.to_dict()]
prompts = [
    """ You are part of the recommendations team. As a helpful assistant, aid in suggesting the colours for the wallpaper generator app, based on the user's preferences given. """
    """Suggest new wallpaper colours for the user based on the given preferences. Apply the guidlines as mentioned below, do not hallucinate the answer."""
    """The user preference is a table with their preferences denoted by like/dislike, and it is as follows:    userAction  c1r  c1g  c1b  c2r  c2g  c2b  c3r  c3g  c3b  c4r  c4g  c4b
0     dislike  193   19   59  185   86  143  250   55  218  152  113   12
1        Like   38  154  243   21  189  124  107  103  118   63  156  217
2     dislike   51   66  187   97   15   69  241  145  184    6  218  168
3     dislike  224  171  164  198  238   70  137  127   55  210   11   49
4        Like   97  210  199  223  155   37  186   42   61   53  163  242
    """
    """
    Guidelines:
    1. Give a single colour combination output. Try not to give any already mentioned colour from the preference data.
    3. The output should be given as follows:
        c1r : ## | c1g : ## | c1b: ## | respective colour name | respective colour hex-code
        c2r : ## | c2g : ## | c2b: ## | respective colour name | respective colour hex-code
        c3r : ## | c3g : ## | c3b: ## | respective colour name | respective colour hex-code
        c4r : ## | c4g : ## | c4b: ## | respective colour name | respective colour hex-code
    4. Finally give a message suggesting a creative name for the wallpaper based on the colours said by you.
    """,

        """ You are part of the recommendations team. As a helpful assistant, aid in suggesting the colours for the wallpaper generator app, based on the user's preferences given. """
    """Suggest new wallpaper colours for the user based on the given preferences. Apply the guidlines as mentioned below, do not hallucinate the answer."""
    """The user preference is a table with their preferences denoted by like/dislike, and it is as follows:    userAction	c1r	c1g	c1b	c2r	c2g	c2b	c3r	c3g	c3b	c4r	c4g	c4b
0	dislike	120	26	189	120	115	204	232	2	102	197	199	154
1	dislike	61	164	224	50	233	171	151	206	58	117	159	95
2	Like	232	179	112	61	240	185	51	11	253	38	129	130
3	dislike	100	112	183	80	186	112	1	129	219	53	86	228
4	Like	224	128	146	125	129	52	171	217	159	197	159	246
    """
    """
    Guidelines:
    1. Give a single colour combination output. Try not to give any already mentioned colour from the preference data.
    3. The output should be given as follows:
        c1r : ## | c1g : ## | c1b: ## | respective colour name | respective colour hex-code
        c2r : ## | c2g : ## | c2b: ## | respective colour name | respective colour hex-code
        c3r : ## | c3g : ## | c3b: ## | respective colour name | respective colour hex-code
        c4r : ## | c4g : ## | c4b: ## | respective colour name | respective colour hex-code
    4. Finally give a message suggesting a creative name for the wallpaper based on the colours said by you.
    """,

    """ You are part of the recommendations team. As a helpful assistant, aid in suggesting the colours for the wallpaper generator app, based on the user's preferences given. """
    """Suggest new wallpaper colours for the user based on the given preferences. Apply the guidlines as mentioned below, do not hallucinate the answer."""
    """The user preference is a table with their preferences denoted by like/dislike, and it is as follows:    	userAction	c1r	c1g	c1b	c2r	c2g	c2b	c3r	c3g	c3b	c4r	c4g	c4b
0	Like	130	0	4	217	246	254	141	102	26	136	206	14
1	Like	41	123	204	178	62	95	230	240	51	252	95	131
2	Like	228	150	230	236	142	170	28	35	12	159	70	186
3	dislike	85	27	65	169	44	61	184	244	133	27	27	107
4	Like	83	29	189	74	127	249	246	91	216	230	189	224
    """
    """
    Guidelines:
    1. Give a single colour combination output. Try not to give any already mentioned colour from the preference data.
    3. The output should be given as follows:
        c1r : ## | c1g : ## | c1b: ## | respective colour name | respective colour hex-code
        c2r : ## | c2g : ## | c2b: ## | respective colour name | respective colour hex-code
        c3r : ## | c3g : ## | c3b: ## | respective colour name | respective colour hex-code
        c4r : ## | c4g : ## | c4b: ## | respective colour name | respective colour hex-code
    4. Finally give a message suggesting a creative name for the wallpaper based on the colours said by you.
    """,

        """ You are part of the recommendations team. As a helpful assistant, aid in suggesting the colours for the wallpaper generator app, based on the user's preferences given. """
    """Suggest new wallpaper colours for the user based on the given preferences. Apply the guidlines as mentioned below, do not hallucinate the answer."""
    """The user preference is a table with their preferences denoted by like/dislike, and it is as follows:    userAction	c1r	c1g	c1b	c2r	c2g	c2b	c3r	c3g	c3b	c4r	c4g	c4b
0	dislike	107	54	243	63	248	130	228	50	134	20	72	166
1	Like	131	88	59	13	241	249	8	89	52	129	83	91
2	dislike	187	198	171	252	7	174	34	205	80	163	49	103
3	Like	1	253	133	53	105	3	53	220	190	145	217	43
4	Like	201	189	227	13	94	47	14	199	205	214	251	248
    """
    """
    Guidelines:
    1. Give a single colour combination output. Try not to give any already mentioned colour from the preference data.
    3. The output should be given as follows:
        c1r : ## | c1g : ## | c1b: ## | respective colour name | respective colour hex-code
        c2r : ## | c2g : ## | c2b: ## | respective colour name | respective colour hex-code
        c3r : ## | c3g : ## | c3b: ## | respective colour name | respective colour hex-code
        c4r : ## | c4g : ## | c4b: ## | respective colour name | respective colour hex-code
    4. Finally give a message suggesting a creative name for the wallpaper based on the colours said by you.
    """,

        """ You are part of the recommendations team. As a helpful assistant, aid in suggesting the colours for the wallpaper generator app, based on the user's preferences given. """
    """Suggest new wallpaper colours for the user based on the given preferences. Apply the guidlines as mentioned below, do not hallucinate the answer."""
    """The user preference is a table with their preferences denoted by like/dislike, and it is as follows:    userAction	c1r	c1g	c1b	c2r	c2g	c2b	c3r	c3g	c3b	c4r	c4g	c4b
0	Like	39	212	207	236	81	110	52	23	153	216	251	187
1	Like	236	40	156	14	44	64	88	70	8	87	128	235
2	Like	215	62	138	242	80	135	162	162	32	122	4	233
3	dislike	249	40	27	134	200	71	11	161	32	47	246	150
4	Like	215	36	98	171	103	213	218	34	192	226	100	174
    """
    """
    Guidelines:
    1. Give a single colour combination output. Try not to give any already mentioned colour from the preference data.
    3. The output should be given as follows:
        c1r : ## | c1g : ## | c1b: ## | respective colour name | respective colour hex-code
        c2r : ## | c2g : ## | c2b: ## | respective colour name | respective colour hex-code
        c3r : ## | c3g : ## | c3b: ## | respective colour name | respective colour hex-code
        c4r : ## | c4g : ## | c4b: ## | respective colour name | respective colour hex-code
    4. Finally give a message suggesting a creative name for the wallpaper based on the colours said by you.
    """
    ]

In [ ]:
r1 = {
 'userAction': {0: 'dislike'},
 'c1r': {0: 100},
 'c1g': {0: 200},
 'c1b': {0: 255},
 'c2r': {0: 255},
 'c2g': {0: 100},
 'c2b': {0: 100},
 'c3r': {0: 200},
 'c3g': {0: 200},
 'c3b': {0: 255},
 'c4r': {0: 100},
 'c4g': {0: 100},
 'c4b': {0: 255}
}

r2 = {
 'userAction': {0: 'dislike'},
 'c1r': {0: 240},
 'c1g': {0: 185},
 'c1b': {0: 51},
 'c2r': {0: 125},
 'c2g': {0: 129},
 'c2b': {0: 52},
 'c3r': {0: 171},
 'c3g': {0: 217},
 'c3b': {0: 159},
 'c4r': {0: 159},
 'c4g': {0: 197},
 'c4b': {0: 159}
}

r3 = {
 'userAction': {0: 'like'},
 'c1r': {0: 178},
 'c1g': {0: 62},
 'c1b': {0: 95},
 'c2r': {0: 236},
 'c2g': {0: 142},
 'c2b': {0: 170},
 'c3r': {0: 228},
 'c3g': {0: 150},
 'c3b': {0: 230},
 'c4r': {0: 83},
 'c4g': {0: 29},
 'c4b': {0: 189}
}

r4 = {
 'userAction': {0: 'like'},
 'c1r': {0: 240},
 'c1g': {0: 240},
 'c1b': {0: 150},
 'c2r': {0: 150},
 'c2g': {0: 100},
 'c2b': {0: 20},
 'c3r': {0: 200},
 'c3g': {0: 255},
 'c3b': {0: 200},
 'c4r': {0: 255},
 'c4g': {0: 20},
 'c4b': {0: 20}
}

r5 = {
 'userAction': {0: 'like'},
 'c1r': {0: 123},
 'c1g': {0: 234},
 'c1b': {0: 255},
 'c2r': {0: 255},
 'c2g': {0: 204},
 'c2b': {0: 153},
 'c3r': {0: 255},
 'c3g': {0: 102},
 'c3b': {0: 0},
 'c4r': {0: 255},
 'c4g': {0: 0},
 'c4b': {0: 128}
}

In [ ]:
rejected = [r1, r2, r3, r4, r5]

In [ ]:
data = pd.DataFrame()
data['prompt'] = prompts
data['chosen'] = chosen
data['rejected'] = rejected

In [ ]:
data

,prompt,chosen,rejected
0,You are part of the recommendations team. As ...,"{'userAction': {0: 'dislike', 1: 'Like', 2: 'd...","{'userAction': {0: 'dislike'}, 'c1r': {0: 100}..."
1,You are part of the recommendations team. As ...,"{'clickX': {0: 1211.1004452102018, 1: 1124.192...","{'userAction': {0: 'dislike'}, 'c1r': {0: 240}..."
2,You are part of the recommendations team. As ...,"{'clickX': {0: 748.2682201121977, 1: 1958.9429...","{'userAction': {0: 'like'}, 'c1r': {0: 178}, '..."
3,You are part of the recommendations team. As ...,<bound method DataFrame.to_dict of c...,"{'userAction': {0: 'like'}, 'c1r': {0: 240}, '..."
4,You are part of the recommendations team. As ...,"{'clickX': {0: 193.9191718688944, 1: 142.45627...","{'userAction': {0: 'like'}, 'c1r': {0: 123}, '..."


#### Pushing the created dataset to hub

In [ ]:
import datasets
pref_sample = datasets.Dataset.from_pandas(pd.DataFrame(data=data.astype(str)))

In [ ]:
pref_sample

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 5
})

In [ ]:
pref_sample.push_to_hub("KamalamSivakumar/pref_learning_sample")

NameError: name 'pref_sample' is not defined

#### Loading the dataset from the huggingface hub.

In [ ]:
from datasets import load_dataset
pref_data = load_dataset("KamalamSivakumar/pref_learning_sample")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
pref_data

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 5
    })
})

### Session Crashes here every single time

In [ ]:
from transformers import AutoConfig, AutoModelForCausalLM
from transformers import AutoTokenizer
import torch
torch.cuda.empty_cache()

# Check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer_id = "mistralai/Mistral-7B-Instruct-v0.3"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
tokenizer.padding_side = 'right' # to prevent warnings

# Load model without device_map and then move it to device manually
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
)
model.to(device)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import psutil

print(f"Available Memory: {psutil.virtual_memory().available / 1e9} GB")

import trl
print(f"TRL Version: {trl.__version__}")

import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices=false'

Available Memory: 325.977182208 GB
TRL Version: 0.9.6


In [ ]:
from trl import ORPOConfig, ORPOTrainer

# try:
#     from trl import ORPOConfig, ORPOTrainer
#     print("Imports successful.")
# except Exception as e:
#     print(f"Import error: {e}")

# Ensure pref_data['train'] is properly defined and available
train_dataset = pref_data['train']

orpo_config = ORPOConfig(
    beta=0.1, # the lambda/alpha hyperparameter in the paper/code
    output_dir="orpo_results/"
)

orpo_trainer = ORPOTrainer(
    model=model,
    args=orpo_config,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/orpo_trainer.py:209: UserWarning: `max_length` is not set in the ORPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/orpo_trainer.py:218: UserWarning: `max_prompt_length` is not set in the ORPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/orpo_trainer.py:247: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
messages = [
    {"role" : "system", "content": system_prompt}, # I have used the 3rd prompt from the pref_data
    {"role": "user", "content": """Suggest new wallpaper colours for the user based on the given preferences. Apply the guidlines as mentioned below, do not hallucinate the answer."""
    """The user preference is a table with their preferences denoted by like/dislike, and it is as follows:    	userAction	c1r	c1g	c1b	c2r	c2g	c2b	c3r	c3g	c3b	c4r	c4g	c4b
0	Like	130	0	4	217	246	254	141	102	26	136	206	14
1	Like	41	123	204	178	62	95	230	240	51	252	95	131
2	Like	228	150	230	236	142	170	28	35	12	159	70	186
3	dislike	85	27	65	169	44	61	184	244	133	27	27	107
4	Like	83	29	189	74	127	249	246	91	216	230	189	224
    """
    """
    Guidelines:
    1. Give a single colour combination output. Try not to give any already mentioned colour from the preference data.
    3. The output should be given as follows:
        c1r : ## | c1g : ## | c1b: ## | respective colour name | respective colour hex-code
        c2r : ## | c2g : ## | c2b: ## | respective colour name | respective colour hex-code
        c3r : ## | c3g : ## | c3b: ## | respective colour name | respective colour hex-code
        c4r : ## | c4g : ## | c4b: ## | respective colour name | respective colour hex-code
    4. Finally give a message suggesting a creative name for the wallpaper based on the colours said by you.
    """}]

with torch.no_grad():
    token_ids = tokenizer.apply_chat_template(messages, return_tensors="pt")
    output_ids = model.generate(
        token_ids.to(model.device),
        temperature=0.1,
        do_sample=True,
        top_p=0.95,
        top_k=40,
        max_new_tokens=1000,
    )
output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
print(output)

In [ ]:
print(u3.to_string())

  userAction  c1r  c1g  c1b  c2r  c2g  c2b  c3r  c3g  c3b  c4r  c4g  c4b
0       Like  130    0    4  217  246  254  141  102   26  136  206   14
1       Like   41  123  204  178   62   95  230  240   51  252   95  131
2       Like  228  150  230  236  142  170   28   35   12  159   70  186
3    dislike   85   27   65  169   44   61  184  244  133   27   27  107
4       Like   83   29  189   74  127  249  246   91  216  230  189  224


In [ ]:
u3

,userAction,c1r,c1g,c1b,c2r,c2g,c2b,c3r,c3g,c3b,c4r,c4g,c4b
0,Like,130,0,4,217,246,254,141,102,26,136,206,14
1,Like,41,123,204,178,62,95,230,240,51,252,95,131
2,Like,228,150,230,236,142,170,28,35,12,159,70,186
3,dislike,85,27,65,169,44,61,184,244,133,27,27,107
4,Like,83,29,189,74,127,249,246,91,216,230,189,224
